# K-NN + оценка по кросс-валидации

In [1]:
#=========================================================================================================
#============================================= ЗАДАНИЕ ЕВКЛИДОВОЙ И КОСИНУСНОЙ МЕТРИК ДЛЯ ОБЪЕКТОВ =======
#=========================================================================================================


def euclidean_distance(X, Y):
    """
        ================================================
        returns euclidean distance between X & Y objects
        
        ================================================
        param .... type ............... meaning ........
        ================================================
        X ........ ndarray (float) .... data ...........
        Y ........ ndarray (float) .... data ...........
        ================================================
    """
    X_part = (X * X).sum(axis = 1)[np.newaxis].T
    Y_part = (Y * Y).sum(axis = 1)[np.newaxis]
    XY_part = np.dot(X, Y.T)
    return (-2 * XY_part + X_part + Y_part) ** 0.5

def cosine_distance(X, Y):
    """
        =============================================
        returns cosine distance between X & Y objects
        
        =============================================
        param .... type ............... meaning .....
        =============================================
        X ........ ndarray (float) .... data ........
        Y ........ ndarray (float) .... data ........
        =============================================
    """
    X_part = ((X * X).sum(axis = 1) ** 0.5)[np.newaxis].T 
    Y_part = ((Y * Y).sum(axis = 1) ** 0.5)[np.newaxis]
    XY_part = np.dot(X, Y.T)
    return 1 - (XY_part / X_part / Y_part)


#=========================================================================================================
#============================================================================== K-NN КЛАССИФИКАТОР =======
#=========================================================================================================


class KNNClassifier:
    def __init__(self, k=5, strategy='brute', metric='euclidean', weights=False):
        """
            ===========================================================================================================
            param .............. type .... meaning ....................................................................
            ===========================================================================================================
            k .................. int ..... hyperparameter K ...........................................................
            strategy ........... str ..... 'my_own', 'brute', 'kd_tree' or 'ball_tree' - strategy for neighbours search
            metric ............. str ..... 'euclidean' or 'cosine' - metric for neighbours search .....................
            weights ............ bool .... whether or not use weightened analysis of neighbours .......................
            ===========================================================================================================
        """
        if strategy != 'my_own':
            self.__knn = NearestNeighbors(n_neighbors=k, algorithm=strategy, metric=metric)
        self.k = k
        self.metric = metric
        self.strategy = strategy
        self.tbs = test_block_size
        self.weights = weights
    
    def fit(self, X, y):
        """
            ========================================
            Fits the model ..........................
        
            ========================================
            param ..... type ............... meaning
            ========================================
            X ......... ndarray (float) .... data ..
            y ......... ndarray (int) ...... labels
            ========================================
        """
        if self.strategy != 'my_own':
            self.__knn.fit(X, y)
        self.__X = X
        self.__y = y
    
    def find_kneighbors(self, X, return_distance=True):
        """
            =======================================================================================
            returns inds of train objs - k neighbors for each obj in X ............................
            if return_distance is True, also returns distances for each neighbour for each obj in X
        
            =======================================================================================
            param .............. type ............... meaning .....................................
            =======================================================================================
            X .................. ndarray (float) .... test data ...................................
            return_distance .... bool ............... whether or not return distance from neighbors
            =======================================================================================
        """
        if self.strategy != 'my_own':
            return self.__knn.kneighbors(X, self.k, return_distance=return_distance)
        else:            
            res_d = np.zeros((1, self.k))
            res_i = np.zeros_like(res_d)
            
            for i in range(0, X.shape[0], self.tbs):
            
                if self.metric == 'euclidean':
                    dist = euclidean_distance(X[i: i + self.tbs], self.__X)
                else:
                    dist = cosine_distance(X[i: i + self.tbs], self.__X)
                    
                bres_i = np.argpartition(dist, kth=np.arange(self.k), axis=1)
                bres_i = bres_i[:, :self.k]
                
                bres_d = np.partition(dist, kth=np.arange(self.k), axis=1)
                bres_d = bres_d[:, :self.k]
                
                res_d = np.concatenate((res_d, bres_d));
                res_i = np.concatenate((res_i, bres_i));
                
            res_d = res_d[1::]
            res_i = res_i[1::]
                
            if return_distance:
                return res_d, np.int64(res_i)
            else:
                return np.int64(res_i)
            
    def predict(self, X):       
        """
            ===========================================
            returns prediction labels for each obj in X
        
            ===========================================
            param .... type ............... meaning ...
            ===========================================
            X ........ ndarray (float) .... test data .
            ===========================================
        """
        yset = set(self.__y)
        if self.weights:
            dist, ind = self.find_kneighbors(X, True)
            w = 1 / (dist + 0.00001)
        else:
            ind = self.find_kneighbors(X, False)
         
        kee = self.__y[ind]
        if self.weights:
                    
            raaa = np.zeros((X.shape[0], 1))
            klist = []
            for k in yset:
                aaa = w.copy()
                aaa[kee != k] = 0
                aaa = np.sum(aaa, axis=1)[:, np.newaxis]
                raaa = np.concatenate((raaa, aaa), axis=1)
                klist += [k]
            raaa = raaa[::, 1::]
            daaa = np.argmax(raaa, axis=1)
            vaaa = [klist[ik] for ik in daaa]
            win = vaaa   
                            
        else:
            win = ([[np.bincount(x).argmax()] for x in kee])
    
        return np.array(win).T

In [2]:
#=========================================================================================================
#================================================================================= КРОСС-ВАЛИДАЦИЯ =======
#=========================================================================================================
    
    
def kfold(n, n_folds):
    """
        =======================================
        indexing for cross-validation folds ...

        =======================================
        param ...... type .... meaning ........
        =======================================
        n .......... int ..... num of data objs
        n_folds .... int ..... num of folds ...
        =======================================
    """
    eif = n // n_folds + 1 # elements in folds + 1
    finc = n % n_folds # folds with +1 size
    RESList = []
    arr = np.arange(n)
    
    for i in range(finc):
        mask = (arr >= eif * i) & (arr < eif * (i + 1))
        Test = arr[mask]
        Train = arr[~mask]
        RESList += [(Train, Test)]
        
    for i in range(n_folds - finc):
        mask = (arr >= eif * finc + (eif - 1) * i) & (arr < eif * finc + (eif - 1) * (i + 1))
        Test = arr[mask]
        Train = arr[~mask]
        RESList += [(Train, Test)]

    return RESList

def knn_cross_val_score(X, y, k_list, score='accuracy', cv=None, **kwargs):
    """
        ==================================================================
        cross-validation .................................................

        ==================================================================
        param ...... type .... meaning ...................................
        ==================================================================
        X ......... ndarray (float) .... data ............................
        y ......... ndarray (int) ...... true labels .....................
        k_list .... list (int) ......... list of hyperparameters k to test
        score ..... str ................ Can only be equal to 'accuracy' .
        cv ........ list ............... None or another result of kfold .
        kwargs .... dict ............... args for KNNCllassifier .........
        ==================================================================
    """
    res = {}
    if cv == None:
        cv=kfold(X.shape[0], 3)
        
    for k in k_list:
        res[k] = []  
        
    knn = KNNClassifier(k=k_list[-1], **kwargs)
    yset = set(y)
    for itrain, itest in cv:
        X_train, X_test = X[itrain], X[itest]
        y_train, y_test = y[itrain], y[itest]
        knn.fit(X_train, y_train)
        dist, ind = knn.find_kneighbors(X_test)
        w = 1 / (dist + 1e-5)
        
        for k in k_list:
            kee = y_train[ind[::, :k]]
            if knn.weights:
                
                wo = w[::, :k]
                raaa = np.zeros((X_test.shape[0], 1))
                klist = []
                for ky in yset:
                    aaa = wo.copy()
                    aaa[kee != ky] = 0
                    aaa = np.sum(aaa, axis=1)[:, np.newaxis]
                    raaa = np.concatenate((raaa, aaa), axis=1)
                    klist += [ky]
                raaa = raaa[::, 1::]
                daaa = np.argmax(raaa, axis=1)
                vaaa = [klist[ik] for ik in daaa]
                win = vaaa   
                            
            else:
                win = [np.unique(x)[0] for x in kee]
            
            if score == 'accuracy':
                scores = np.sum(y_test == win) / y_test.shape[0]
            res[k] += [scores]
    return res

# Логистическая регрессия + ГС + СГС

In [3]:
class BaseSmoothOracle:
    def func(self, w):
        return

    def grad(self, w):
        return

    
#=========================================================================================================
#========================================================================= ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ =======
#=========================================================================================================


class BinaryLogistic(BaseSmoothOracle):
    def __init__(self, l2_coef):
        """
            =================================================
            param ...... type ..... meaning .................
            =================================================
            l2_coef .... float .... coef of L2 regularization
            =================================================
        """
        self.l2_coef = l2_coef
     
    def func(self, X, y, w):
        """
            =============================================
            returns log(1 + exp(-y<w,x>)) for each x in X
            (logistic loss function) ....................
        
            =============================================
            param ...... type ................ meaning ..
            =============================================
            X .... ndarray or csr (float) .... data .....
            y .... ndarray (int) ............. labels ...
            w .... ndarray (float) ........... weights ..
            =============================================
        """
        A = X * w
        if type(X) == np.ndarray:
            A = np.sum(A, axis=1)
        A *= y
        A = np.logaddexp(np.zeros_like(A), -A)
        A = np.sum(A)
        A /= X.shape[0]
        A += self.l2_coef / 2 * np.dot(w.T, w)
        return A
        
    def grad(self, X, y, w):
        """
            ===========================================
            returns sigmoid(-y<w,x>)*xy for each x in X
            (grad of logistic loss function) ..........
        
            ===========================================
            param ...... type ................ meaning 
            ===========================================
            X .... ndarray or csr (float) .... data ...
            y .... ndarray (int) ............. labels .
            w .... ndarray (float) ........... weights 
            ===========================================
        """
        if type(X) == np.ndarray:
            A = -X * w * y[:, np.newaxis]
            A = np.sum(A, axis=1)
            A = expit(A)[:, np.newaxis]
            B = - X * y[:, np.newaxis]
            A = B * A
            A = np.sum(A, axis=0)
            A /= X.shape[0]
            A += self.l2_coef * w
        
        else:
            A = -X * w.T * y
            A = expit(A)[:, np.newaxis]
            d = ssp.lil_matrix((y.shape[0], y.shape[0]))
            d.setdiag(y)
            B = -X.T * d
            A = B * A
            A /= X.shape[0]
            A = A.reshape(-1)
            A = A + self.l2_coef * w

        return A

In [4]:
#=========================================================================================================
#======================================================================= МЕТОД ГРАДИЕНТНОГО СПУСКА =======
#=========================================================================================================


class GDClassifier:
    def __init__(self, loss_function, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, **kwargs):
        """
            ============================================================================================
            GD iteration: parameter := parameter - step_alpha / (step_beta) ^ iter_num * grad(loss_func)
            
            ============================================================================================
            param ............ type ..... meaning ......................................................
            ============================================================================================
            loss_function .... str ...... loss func for GD .............................................
            step_alpha ....... float .... coef of grad shift ...........................................
            step_beta ........ float .... coef of grad shift ...........................................
            tolerance ........ float .... min difference of near losses to continue.....................
            max_iter ......... int ...... max iterations of method .....................................
            kwargs ........... dict ..... args for loss class ..........................................
            ============================================================================================
        """
        if loss_function == 'binary_logistic':
            self.oracle = BinaryLogistic(**kwargs)
        self.alpha = step_alpha
        self.beta = step_beta
        self.tolerance = tolerance
        self.max_iter = max_iter 
        
        
    def fit(self, X, y, Xt=None, yt=None, w_0=None, trace=False, acc=False):
        """
            =================================================================================================
            fits the model (GD iterations), returns history of training (time, loss_func, accs for each iter)
            
            =================================================================================================
            param .... type ....................... meaning .................................................
            =================================================================================================
            X ........ ndarray (float) ............ data ....................................................
            y ........ ndarray (int) .............. labels ..................................................
            Xt ....... None or ndarray (float) .... test data ...............................................
            yt ....... None or ndarray (int) ...... test labels .............................................
            w_0 ...... mdarray (float) ............ start weights ...........................................
            trace .... bool ....................... whether or not to measure iters times and loss_func .....
            acc ...... bool ....................... whether or not to measure iters accuracy ................
            =================================================================================================
        """
        self.labels = np.unique(y)
        if type(w_0) == np.ndarray:
            self.w = w_0
        else:
            self.w = np.zeros(X.shape[1])
        tol_flag = False
        iters = 1
        loss_1 = self.oracle.func(X, y, self.w)
        if trace:
            history = {}
            history['time'] = [0]
            history['func'] = [loss_1]
            if acc == True:
                history['acc'] = [np.sum(self.predict(Xt) == yt) / len(yt)]
        while ((not(tol_flag)) & (iters < self.max_iter + 1)):
            
            if trace:
                start_iter = time.time()
            grad = self.oracle.grad(X, y, self.w)
            self.w -= self.alpha / np.power(iters, self.beta) * grad
            loss_2 = self.oracle.func(X, y, self.w)
            if abs(loss_1 - loss_2) < self.tolerance:
                tol_flag = True
            loss_1 = loss_2
            if trace:
                end_iter = time.time()
                history['time'] += [end_iter - start_iter]
                history['func'] += [loss_1]
                if acc == True:
                    history['acc'] += [np.sum(self.predict(Xt) == yt) / len(yt)]
            iters += 1
        if trace:
            return history
        
    def predict(self, X):
        """
            =======================================
            predicts y for each x in X
            
            =======================================
            param .... type ............... meaning
            =======================================
            X ........ ndarray (float) .... data ..
            =======================================
        """
        A = X * self.w
        if type(X) == np.ndarray:
            A = np.sum(A, axis=1) 
        A = np.sign(A)
        return A

    def predict_proba(self, X):
        """
            ===============================================================
            predicts propability of (y == +1) and (y == -1) for each x in X
            
            ===============================================================
            param .... type ............... meaning .......................
            ===============================================================
            X ........ ndarray (float) .... data ..........................
            ===============================================================
        """
        A = np.zeros((X.shape[0], self.labels.shape[0]))
        B = X * self.w
        if type(X) == np.ndarray:
            B = np.sum(B, axis=1)
        A[:, 0] = expit(B)
        A[:, 1] = expit(-B)
        return A
        
        
#=========================================================================================================
#=============================================================================== СЛУЖЕБНЫЕ ФУНКЦИИ =======
#=========================================================================================================


    def get_objective(self, X, y):
        return self.oracle.func(X, y, self.w)
        
    def get_gradient(self, X, y):
        return self.oracle.grad(X, y, self.w)
    
    def get_weights(self):
        return self.w

In [5]:
#=========================================================================================================
#======================================================= МЕТОД СТОХАСТИЧЕСКОГО ГРАДИЕНТНОГО СПУСКА =======
#=========================================================================================================


class SGDClassifier(GDClassifier):
    def __init__(self, loss_function, batch_size, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, random_seed=153, **kwargs):
        """
            =======================================================================================================
            SGD iteration: parameter := parameter - step_alpha / (step_beta) ^ iter_num * grad(loss_func_for_batch)
            predict & predict_proba inherited from GDClassifier ...................................................

            =======================================================================================================
            param ............ type ..... meaning .................................................................
            =======================================================================================================
            loss_function .... str ...... loss func for SGD .......................................................
            batch_size ....... int ...... size of batch ...........................................................
            step_alpha ....... float .... coef of grad shift ......................................................
            step_beta ........ float .... coef of grad shift ......................................................
            tolerance ........ float .... min difference of near losses to continue................................
            max_iter ......... int ...... max iterations of method ................................................
            random_seed ...... int ...... seed for data permutation ...............................................
            kwargs ........... dict ..... args for loss class .....................................................
            =======================================================================================================
        """
        super().__init__(loss_function, step_alpha, step_beta, tolerance, max_iter, **kwargs)
        self.rs = random_seed
        self.bs = batch_size
        pass
        
    def fit(self, X, y, Xt=None, yt=None, w_0=None, trace=False, acc=False, log_freq=1):
        """
            ==================================================================================================
            fits the model (SGD iterations), returns history of training (time, loss_func, accs for each iter)
            
            ==================================================================================================
            param ....... type ....................... meaning ...............................................
            ==================================================================================================
            X ........... ndarray (float) ............ data ..................................................
            y ........... ndarray (int) .............. labels ................................................
            Xt .......... None or ndarray (float) .... test data .............................................
            yt .......... None or ndarray (int) ...... test labels ...........................................
            w_0 ......... mdarray (float) ............ start weights .........................................
            trace ....... bool ....................... whether or not to measure iters times and loss_func ...
            acc ......... bool ....................... whether or not to measure iters accuracy ..............
            log_freq .... float ...................... fraction of using data for one epoch ..................
            ==================================================================================================
        """
        np.random.seed(self.rs)
        permuts = np.random.permutation(np.arange(len(y)))
        
        if type(w_0) == np.ndarray:
            self.w = w_0
        else:
            self.w = np.zeros(X.shape[1])
        tol_flag = False
        iters = 1
        loss_1 = self.oracle.func(X, y, self.w)
        
        if trace:
            history = {}
            history['epoch_num'] = [0]
            history['time'] = [0]
            history['func'] = [loss_1]
            history['weights_diff'] = [0]
            if acc == True:
                history['acc'] = [np.sum(self.predict(Xt) == yt) / len(yt)]
         
        while ((not(tol_flag)) & (iters < self.max_iter + 1)):
            permuts = np.random.permutation(np.arange(len(y)))
            w_diff = np.zeros_like(self.w)
            if trace:
                start_iter = time.time()
            ind = 0
            look_el = 0
            while (ind < len(y)) & ((look_el / len(y)) < log_freq):
                index_loc = permuts[ind : ind + self.bs]
                X_loc = X[index_loc]
                y_loc = y[index_loc]
                grad = self.oracle.grad(X_loc, y_loc, self.w)
                w_diff -= self.alpha / np.power(iters, self.beta) * grad
                self.w -= self.alpha / np.power(iters, self.beta) * grad
                ind += self.bs
                look_el += self.bs
            
            loss_2 = self.oracle.func(X, y, self.w)
            if np.abs(loss_1 - loss_2) < self.tolerance:
                tol_flag = True
            loss_1 = loss_2
            if trace:
                end_iter = time.time()
                history['epoch_num'] += [iters]
                history['time'] += [end_iter - start_iter]
                history['func'] += [loss_1]
                history['weights_diff'] += [np.dot(w_diff.T, w_diff)]
                if acc == True:
                    history['acc'] += [np.sum(self.predict(Xt) == yt) / len(yt)]
            iters += 1
        if trace:
            return history

# Случайный лес + градиентный бустинг

In [6]:
#=========================================================================================================
#=================================================================================== СЛУЧАЙНЫЙ ЛЕС =======
#=========================================================================================================


class RandomForestMSE:
    def __init__(self, n_estimators, max_depth=None, feature_subsample_size=None,
                 **trees_parameters):
        """
            ===================================================================================
            Random Forest (as prediction returns forest mean) .................................

            ===================================================================================
            param ..................... type ........... meaning ..............................
            ===================================================================================
            n_estimators .............. int ............ trees number .........................
            max_depth ................. int ............ max depth for each tree ..............
            feature_subsample_size .... None or int .... volume of using features for each tree
            trees_parameters .......... dict ........... args for tree construction  ..........
            ===================================================================================
        """
        if feature_subsample_size is None:
            self.fs = 'Recommended'
        else:
            self.fs = 'Choosen'
            self.f = feature_subsample_size
        self.n = n_estimators
        
        self.trees = {}
        for i in range(self.n):
            self.trees[i] = DecisionTreeRegressor(max_depth=max_depth, **trees_parameters)
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
            ======================================================
            fits the model (each tree)  ..........................
            if X_val is not None, saves history of training (RMSE)
            
            ======================================================
            param .... type ....................... meaning ......
            ======================================================
            X ........ ndarray (float) ............ data .........
            y ........ ndarray (float) ............ responses ....
            X_val .... None or ndarray (float) .... test data ....
            y_val .... None or ndarray (float) .... test responses
            ======================================================
        """
        if self.fs == 'Recommended':
            self.f = X.shape[1] // 3
            
        self.featuress = np.zeros((self.f, self.n))
        if not X_val is None:
            self.RMSE = []
        else:
            self.RMSE = None
            
        for i in range(self.n):
            train_sample = np.random.randint(0, X.shape[0], X.shape[0])
            features = np.random.choice(range(X.shape[1]), self.f, replace=False)
            self.featuress[:, i] = features
            X_train_local = X[train_sample, :][:, features]
            y_train_local = y[train_sample]
            self.trees[i].fit(X_train_local, y_train_local)
            if not X_val is None:
                self.RMSE += [np.sqrt(mean_squared_error(y_val, self.predict(X_val, i + 1)))]
        
    def predict(self, X, N=None):
        """
            ========================================================
            predicts y for each x in X .............................
            N regularize volume of trained forest ..................
            
            ========================================================
            param .... type ............... meaning ................
            ========================================================
            X ........ ndarray (float) .... data ...................
            N ........ int or None ........ volume of forest to test
            ========================================================
        """
        if N == None or N > self.n:
            N = self.n
        y_preds = np.zeros((N, X.shape[0]))
        for i in range(N):
            y_preds[i, :] = self.trees[i].predict(X[:, self.featuress[:, i].astype(int)])
        y_pred_res = np.mean(y_preds, axis=0)
        return y_pred_res

In [7]:
#=========================================================================================================
#============================================================================= ГРАДИЕНТНЫЙ БУСТИНГ =======
#=========================================================================================================
    

class GradientBoostingMSE:
    def __init__(self, n_estimators, learning_rate=0.1, max_depth=5, feature_subsample_size=None,
                 **trees_parameters):
        """
            ====================================================================================
            param ...................... type ........... meaning ..............................
            ====================================================================================
            n_estimators ............... int ............ trees number .........................
            learning_rate .............. float .......... learnng rate in models combination ...
            max_depth .................. int ............ max depth for each tree ..............
            feature_subsample_size ....  None or int .... volume of using features for each tree
            trees_parameters ........... dict ........... args for tree construction  ..........
            ====================================================================================
        """
        if feature_subsample_size is None:
            self.fs = 'Recommended'
        else:
            self.fs = 'Choosen'
            self.f = feature_subsample_size
        self.n = n_estimators
        self.l = learning_rate
        self.algs = {}
        for i in range(1, self.n):
            self.algs[i] = DecisionTreeRegressor(max_depth=max_depth, **trees_parameters)
        
    def fit(self, X, y, X_val=None, y_val=None):
        """
            ======================================================
            fits the model (each tree)  ..........................
            if X_val is not None, saves history of training (RMSE)
            
            ======================================================
            param .... type ....................... meaning ......
            ======================================================
            X ........ ndarray (float) ............ data .........
            y ........ ndarray (float) ............ responses ....
            X_val .... None or ndarray (float) .... test data ....
            y_val .... None or ndarray (float) .... test responses
            ======================================================
        """
        if self.fs == 'Recommended':
            self.f = X.shape[1] // 3
        
        self.featuress = np.zeros((self.f, self.n))
        train_sample = np.random.randint(0, X.shape[0], X.shape[0])
        y_train_local = y[train_sample]
        self.start = np.mean(y_train_local)
        self.algs[0] = lambda X: self.start
        self.alphas = [1]
        if not X_val is None:
            self.RMSE = []
            self.RMSE += [np.sqrt(mean_squared_error(y_val, self.predict(X_val, 1)))]
        else:
            self.RMSE = None

        for i in range(1, self.n):
            train_sample = np.random.randint(0, X.shape[0], X.shape[0])
            features = np.random.choice(range(X.shape[1]), self.f, replace=False)
            self.featuress[:, i] = features
            X_train_local = X[train_sample, :][:, features]
            y_train_local = y[train_sample].reshape(-1)
            curr_alg_res = self.algs[0](X_train_local)
            for j in range(1, i):
                curr_alg_res += self.l * self.alphas[j] * self.algs[j].predict(X[train_sample, :][:, self.featuress[:, j].astype(int)])
            rs = -2 * (curr_alg_res - y_train_local)
            self.algs[i].fit(X_train_local, rs)
            
            new_alg = lambda alpha: np.sum((curr_alg_res + alpha * self.algs[i].predict(X_train_local) - y_train_local) ** 2)
            best_alpha = minimize_scalar(new_alg).x
            self.alphas += [best_alpha]
            
            if not X_val is None:
                self.RMSE += [np.sqrt(mean_squared_error(y_val, self.predict(X_val, i + 1)))]

    def predict(self, X, N=None):
        """
            ========================================================
            predicts y for each x in X .............................
            N regularize volume of trained forest ..................
            
            ========================================================
            param .... type ............... meaning ................
            ========================================================
            X ........ ndarray (float) .... data ...................
            N ........ int or None ........ volume of forest to test
            ========================================================
        """
        if N == None or N >= self.n:
            N = self.n
        res = np.ones(X.shape[0]) * self.algs[0](X)
        
        for i in range(1, N):
            a = self.featuress[:, i].astype(int)
            res += self.l * self.alphas[i] * self.algs[i].predict(X[:, a])
            
        return res

# Random Fourier Features + Orthogonal Random Features

In [8]:
#=========================================================================================================
#========================================================================= RANDOM FOURIER FEATURES =======
#=========================================================================================================


from sklearn.base import BaseEstimator, TransformerMixin
class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """
            ====================================================================================================
            Implements pipeline, which consists of PCA decomposition ...........................................
            Random Fourier Features approximation and linear classification model ..............................
        
            ====================================================================================================
            param ......... type .... meaning ..................................................................
            ====================================================================================================
            n_features .... int ..... amount of synthetic random features generated with RFF approximation .....
            new_dim ....... int ..... PCA output size ..........................................................
            use_PCA ....... int ..... whether to include PCA preprocessin ......................................
            classifier .... bool .... 'svm' or 'logreg', a linear classification model to use on top of pipeline
            ====================================================================================================
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
            =====================================================================
            Fit all parts of algorithm (PCA, RFF, Classification) to training set
            returns train time ..................................................
            
            =====================================================================
            param .... type ............... meaning .............................
            =====================================================================
            X ........ ndarray (float) .... data ................................
            y ........ ndarray (int) ...... labels ..............................
            =====================================================================
        """
        start_time = time.time() # функция в конце работы возвращает время выполнения
        X1 = X.astype(float)
        
        #====== PCA ================================================================
        self.sc1 = StandardScaler()
        X1 = self.sc1.fit_transform(X1) # требуется для достижения заданной точности
        if self.use_PCA:
            self.pca = PCA(n_components=self.new_dim)
            X2 = self.pca.fit_transform(X1)
        else:
            X2 = copy.copy(X1)
        #===========================================================================
        
        
        #===== RFF =================================================================
        first_obj = np.random.choice(X2.shape[0], int(1e6))
        second_obj = np.random.choice(X2.shape[0], int(1e6))
        similarity = len(np.where(first_obj == second_obj)[0])
        while similarity != 0: # similarity - число совпадающих объектов для оценки квадрата сигмы, этот цикл нужен для обнуления similarity (чтобы избежать смещения оценки)
            second_obj[second_obj == first_obj] = np.random.choice(X2.shape[0], similarity)
            similarity = len(np.where(first_obj == second_obj)[0])
        sigma2 = np.median(np.sum((X2[first_obj] - X2[second_obj]) ** 2, axis=1))
        
        self.W = np.random.normal(0, 1/sigma2, (X2.shape[1], self.n_features))
        self.b = np.random.uniform(-math.pi, math.pi, (self.n_features))
        self.sc2 = StandardScaler()
        X3 = self.sc2.fit_transform(np.cos(np.matmul(X2, self.W) + self.b))
        #===========================================================================
        
        
        #===== Model's fit =========================================================
        if self.classifier == 'logreg':
            self.model = LogisticRegression(max_iter=200).fit(X3, y)
        else:
            self.model = LinearSVC(max_iter=250).fit(X3, y)
        #===========================================================================
        return time.time() - start_time

    def predict_proba(self, X):
        """
            ==============================================
            Apply pipeline to obtain scores for input data
            
            ==============================================
            param .... type ............... meaning ......
            ==============================================
            X ........ ndarray (float) .... data .........
            ==============================================
        """
        X1 = X.astype(float)
        X1 = self.sc1.transform(X1)
        if self.use_PCA:
            X2 = self.pca.transform(X)
        else:
            X2 = copy.copy(X)
        X3 = self.sc1.transform(np.cos(np.matmul(X2, self.W) + self.b))
        if self.classifier == 'logreg':
            return self.model.predict_proba(X3)
        else:
            return scipy.expit(self.model.desicion_function(X3))
        
    def predict(self, X):
        """
            ============================================================
            Apply pipeline to obtain discrete predictions for input data
            
            ============================================================
            param .... type ............... meaning ....................
            ============================================================
            X ........ ndarray (float) .... data .......................
            ============================================================
        """
        X1 = X.astype(float)
        X1 = self.sc1.transform(X1)
        if self.use_PCA:
            X2 = self.pca.transform(X1)
        else:
            X2 = copy.copy(X1)  
        X3 = self.sc2.transform(np.cos(np.matmul(X2, self.W) + self.b))
        return self.model.predict(X3)

In [9]:
#=========================================================================================================
#====================================================================== ORTHOGONAL RANDOM FEATURES =======
#=========================================================================================================


from sklearn.base import BaseEstimator, TransformerMixin
class ORFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """
            ====================================================================================================
            Implements pipeline, which consists of PCA decomposition ...........................................
            Orthogonal Random Features approximation and linear classification model ...........................
        
            ====================================================================================================
            param ......... type .... meaning ..................................................................
            ====================================================================================================
            n_features .... int ..... amount of synthetic random features generated with ORF approximation .....
            new_dim ....... int ..... PCA output size ..........................................................
            use_PCA ....... int ..... whether to include PCA preprocessin ......................................
            classifier .... bool .... 'svm' or 'logreg', a linear classification model to use on top of pipeline
            ====================================================================================================
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
            =====================================================================
            Fit all parts of algorithm (PCA, ORF, Classification) to training set
            returns train time ..................................................
            
            =====================================================================
            param .... type ............... meaning .............................
            =====================================================================
            X ........ ndarray (float) .... data ................................
            y ........ ndarray (int) ...... labels ..............................
            =====================================================================
        """
        start_time = time.time() # функция в конце работы возвращает время выполнения
        X1 = X.astype(float)
        
        #====== PCA ================================================================
        self.sc1 = StandardScaler()
        X1 = self.sc1.fit_transform(X1) # требуется для достижения заданной точности
        if self.use_PCA:
            self.pca = PCA(n_components=self.new_dim)
            X2 = self.pca.fit_transform(X1)
        else:
            X2 = copy.copy(X1)
        #===========================================================================
        
        
        #===== ORF =================================================================
        first_obj = np.random.choice(X2.shape[0], int(1e6))
        second_obj = np.random.choice(X2.shape[0], int(1e6))
        similarity = len(np.where(first_obj == second_obj)[0])
        while similarity != 0: # similarity - число совпадающих объектов для оценки квадрата сигмы, этот цикл нужен для обнуления similarity (чтобы избежать смещения оценки)
            second_obj[second_obj == first_obj] = np.random.choice(X2.shape[0], similarity)
            similarity = len(np.where(first_obj == second_obj)[0])
        sigma2 = np.median(np.sum((X2[first_obj] - X2[second_obj]) ** 2, axis=1))
        
        d = X2.shape[1]
        G = np.random.normal(0, 1/sigma2, (self.n_features, d))
        Q, R = np.linalg.qr(G, mode='complete')
        Q = Q[:d, :self.n_features]
        self.W = Q
        self.b = np.random.uniform(-math.pi, math.pi, (self.n_features))
        self.sc2 = StandardScaler()
        X3 = self.sc2.fit_transform(np.cos(np.matmul(X2, self.W) + self.b))
        #===========================================================================
        
        
        #===== Model's fit =========================================================
        if self.classifier == 'logreg':
            self.model = LogisticRegression(max_iter=200).fit(X3, y)
        else:
            self.model = LinearSVC(max_iter=50).fit(X3, y)
        #===========================================================================
        return time.time() - start_time

    def predict_proba(self, X):
        """
            ==============================================
            Apply pipeline to obtain scores for input data
            
            ==============================================
            param .... type ............... meaning ......
            ==============================================
            X ........ ndarray (float) .... data .........
            ==============================================
        """
        X1 = X.astype(float)
        X1 = self.sc1.transform(X1)
        if self.use_PCA:
            X2 = self.pca.transform(X)
        else:
            X2 = copy.copy(X)
        X3 = self.sc1.transform(np.cos(np.matmul(X2, self.W) + self.b))
        if self.classifier == 'logreg':
            return self.model.predict_proba(X3)
        else:
            return scipy.expit(self.model.desicion_function(X3))
        
    def predict(self, X):
        """
            ============================================================
            Apply pipeline to obtain discrete predictions for input data
            
            ============================================================
            param .... type ............... meaning ....................
            ============================================================
            X ........ ndarray (float) .... data .......................
            ============================================================
        """
        X1 = X.astype(float)
        X1 = self.sc1.transform(X1)
        if self.use_PCA:
            X2 = self.pca.transform(X1)
        else:
            X2 = copy.copy(X1)  
        X3 = self.sc2.transform(np.cos(np.matmul(X2, self.W) + self.b))
        return self.model.predict(X3)

# EM-алгоритм в задаче перевода (Word Alignment)

Пусть $S=(s_1,\ldots,s_n)$ исходное предложение, $T=(t_1,\ldots,t_m)$ — его перевод. В роли латентных переменных будут выступать выравнивания $A=(a_1,\ldots,a_m)$ каждого слова в целевом предложении, причём $a_i\in\{1,\ldots,n\}$ (считаем, что каждое слово в $t$ является переводом какого-то слова из $s$). Параметрами модели является матрица условных вероятностей перевода: каждый её элемент $\theta(y|x)=p(y|x)$ отражает вероятность того, что переводом слова $x$ с исходного языка на целевой является слово $y$ (нормировка, соответственно, совершается по словарю целевого языка). Правдоподобие латентных переменных и предложения на целевом языке в этой модели записывается так:

$$
p(A,T|S)=\prod_{i=1}^m p(a_i)p(t_i|a_i,S)=\prod_{i=1}^m \frac{1}{n}\theta(t_i|s_{a_i}).
$$ 

In [10]:
from dataclasses import dataclass
from typing import Dict, List, Tuple

from xml.dom import minidom
import xml

import numpy as np


#=========================================================================================================
#================================================================ ВСПОМОГАТЕЛЬНЫЕ СТРУКТУРЫ ДАННЫХ =======
#=========================================================================================================


@dataclass(frozen=True)
class SentencePair:
    """
    Contains lists of tokens (strings) for source and target sentence
    """
    source: List[str]
    target: List[str]


@dataclass(frozen=True)
class TokenizedSentencePair:
    """
    Contains arrays of token vocabulary indices (preferably np.int32) for source and target sentence
    """
    source_tokens: np.ndarray
    target_tokens: np.ndarray


@dataclass(frozen=True)
class LabeledAlignment:
    """
    Contains arrays of alignments (lists of tuples (source_pos, target_pos)) for a given sentence.
    Positions are numbered from 1.
    """
    sure: List[Tuple[int, int]]
    possible: List[Tuple[int, int]]
        
        
#=========================================================================================================
#====================================================================== ПРЕПРОЦЕССИНГ: ТОКЕНИЗАЦИЯ =======
#=========================================================================================================


def extract_sentences(filename: str) -> Tuple[List[SentencePair], List[LabeledAlignment]]:
    """
    Given a file with tokenized parallel sentences and alignments in XML format, return a list of sentence pairs
    and alignments for each sentence.

    Args:
        filename: Name of the file containing XML markup for labeled alignments

    Returns:
        sentence_pairs: list of `SentencePair`s for each sentence in the file
        alignments: list of `LabeledAlignment`s corresponding to these sentences
    """
    with open(filename, encoding="utf8") as f:
        parsing_file = minidom.parseString(f.read().replace('&', '&amp;'))
        sentences = []
        tokens = []
        lvl1 = parsing_file.getElementsByTagName('sentences')[0]
        for lvl2 in lvl1.getElementsByTagName('s'):
            sentences += [SentencePair(lvl2.getElementsByTagName('english')[0].childNodes[0].nodeValue.split(' '),
                                       lvl2.getElementsByTagName('czech')[0].childNodes[0].nodeValue.split(' '))]
            
            sures = lvl2.getElementsByTagName('sure')[0].childNodes
            if len(sures) > 0:
                digits = sures[0].nodeValue.split(' ')
                sures = []
                for d in digits:
                    d0 = d.split('-')
                    sures += [(int(d0[0]), int(d0[1]))]
            else:
                sures = []
                
            poss = lvl2.getElementsByTagName('possible')[0].childNodes
            if len(poss) > 0:
                digits = poss[0].nodeValue.split(' ')
                poss = []
                for d in digits:
                    d0 = d.split('-')
                    poss += [(int(d0[0]), int(d0[1]))]
            else:
                poss = []
            
            tokens += [LabeledAlignment(sures, poss)]
        return sentences, tokens

def get_token_to_index(sentence_pairs: List[SentencePair], freq_cutoff=None) -> Tuple[Dict[str, int], Dict[str, int]]:
    """
    Given a parallel corpus, create two dictionaries token->index for source and target language.

    Args:
        sentence_pairs: list of `SentencePair`s for token frequency estimation
        freq_cutoff: if not None, keep only freq_cutoff most frequent tokens in each language

    Returns:
        source_dict: mapping of token to a unique number (from 0 to vocabulary size) for source language
        target_dict: mapping of token to a unique number (from 0 to vocabulary size) target language

    """
    source_dict = {}
    target_dict = {}
    source_freq = {}
    target_freq = {}
    
    source_reverse = {}
    target_reverse = {}
    source_ind = 0
    target_ind = 0
    for sp in sentence_pairs:
        eng = sp.source
        che = sp.target
        for e in eng:
            if not (e in source_dict):
                source_dict[e] = source_ind
                source_reverse[source_ind] = e
                source_ind += 1
                source_freq[e] = 0
            source_freq[e] += 1
        for c in che:
            if not (c in target_dict):
                target_dict[c] = target_ind
                target_reverse[target_ind] = c
                target_ind += 1
                target_freq[c] = 0
            target_freq[c] += 1
    
    if not (freq_cutoff is None):
        eng_array = np.array([source_freq[x] for x in source_freq])
        che_array = np.array([target_freq[x] for x in target_freq])
        if freq_cutoff > len(eng_array) or freq_cutoff > len(che_array):
            return source_dict, target_dict
        eng_cut = np.argpartition(-eng_array, kth=np.arange(freq_cutoff))[:freq_cutoff]
        che_cut = np.argpartition(-che_array, kth=np.arange(freq_cutoff))[:freq_cutoff]
        new_source_dict = {source_reverse[x]: y for x, y in zip(eng_cut, np.arange(freq_cutoff))}
        new_target_dict = {target_reverse[x]: y for x, y in zip(che_cut, np.arange(freq_cutoff))}
        source_dict = new_source_dict
        target_dict = new_target_dict
            
    return source_dict, target_dict


def tokenize_sents(sentence_pairs: List[SentencePair], source_dict, target_dict) -> List[TokenizedSentencePair]:
    """
    Given a parallel corpus and token_to_index for each language, transform each pair of sentences from lists
    of strings to arrays of integers. If either source or target sentence has no tokens that occur in corresponding
    token_to_index, do not include this pair in the result.
    
    Args:
        sentence_pairs: list of `SentencePair`s for transformation
        source_dict: mapping of token to a unique number for source language
        target_dict: mapping of token to a unique number for target language

    Returns:
        tokenized_sentence_pairs: sentences from sentence_pairs, tokenized using source_dict and target_dict
    """
    res = []
    for sp in sentence_pairs:
        try:
            eng = sp.source
            che = sp.target
            eng_code = np.zeros(len(eng))
            che_code = np.zeros(len(che))
            for eng_word, i in zip(eng, np.arange(len(eng))):
                if eng_word in source_dict:
                    eng_code[i] = source_dict[eng_word]
                else:
                    raise AssertionError
            for che_word, i in zip(che, np.arange(len(che))):
                if che_word in target_dict:
                    che_code[i] = target_dict[che_word]
                else:
                    raise AssertionError
            res += [TokenizedSentencePair(eng_code.astype(np.int32), che_code.astype(np.int32))]
        except:
            pass
    return res


In [11]:
#=========================================================================================================
#================================================================================ МЕТРИКИ КАЧЕСТВА =======
#=========================================================================================================


def compute_precision(reference: List[LabeledAlignment], predicted: List[List[Tuple[int, int]]]) -> Tuple[int, int]:
    """
    Computes the numerator and the denominator of the precision for predicted alignments.
    Numerator : |predicted and possible|
    Denominator: |predicted|
    Note that for correct metric values `sure` needs to be a subset of `possible`, but it is not the case for input data.

    Args:
        reference: list of alignments with fields `possible` and `sure`
        predicted: list of alignments, i.e. lists of tuples (source_pos, target_pos)

    Returns:
        intersection: number of alignments that are both in predicted and possible sets, summed over all sentences
        total_predicted: total number of predicted alignments over all sentences
    """
    intersection = 0
    total_predicted = 0
    for LA, PR in zip(reference, predicted):
        total_predicted += len(PR)
        intersection += len(list((set(LA.possible) | set(LA.sure)) & set(PR)))
    return float(intersection), float(total_predicted)


def compute_recall(reference: List[LabeledAlignment], predicted: List[List[Tuple[int, int]]]) -> Tuple[int, int]:
    """
    Computes the numerator and the denominator of the recall for predicted alignments.
    Numerator : |predicted and sure|
    Denominator: |sure|

    Args:
        reference: list of alignments with fields `possible` and `sure`
        predicted: list of alignments, i.e. lists of tuples (source_pos, target_pos)

    Returns:
        intersection: number of alignments that are both in predicted and sure sets, summed over all sentences
        total_predicted: total number of sure alignments over all sentences
    """
    intersection = 0
    total_predicted = 0
    for LA, PR in zip(reference, predicted):
        total_predicted += len(LA.sure)
        intersection += len(list(set(LA.sure) & set(PR)))
    return float(intersection), float(total_predicted)


def compute_aer(reference: List[LabeledAlignment], predicted: List[List[Tuple[int, int]]]) -> float:
    """
    Computes the alignment error rate for predictions.
    AER=1-(|predicted and possible|+|predicted and sure|)/(|predicted|+|sure|)
    Please use compute_precision and compute_recall to reduce code duplication.

    Args:
        reference: list of alignments with fields `possible` and `sure`
        predicted: list of alignments, i.e. lists of tuples (source_pos, target_pos)

    Returns:
        aer: the alignment error rate
    """
    precision = compute_precision(reference, predicted)
    recall = compute_recall(reference, predicted)
    aer = 1 - (precision[0] + recall[0]) / (precision[1] + recall[1])
    return aer


In [12]:
#=========================================================================================================
#=============================================================== ПРЕДОПИСАННАЯ МОДЕЛЬ ВЫРАВНИВАНИЯ =======
#=========================================================================================================


from abc import ABC, abstractmethod
from itertools import product
class BaseAligner(ABC):
    """
    Describes a public interface for word alignment models.
    """

    @abstractmethod
    def fit(self, parallel_corpus: List[TokenizedSentencePair]):
        """
        Estimate alignment model parameters from a collection of parallel sentences.

        Args:
            parallel_corpus: list of sentences with translations, given as numpy arrays of vocabulary indices

        Returns:
        """
        pass

    @abstractmethod
    def align(self, sentences: List[TokenizedSentencePair]) -> List[List[Tuple[int, int]]]:
        """
        Given a list of tokenized sentences, predict alignments of source and target words.

        Args:
            sentences: list of sentences with translations, given as numpy arrays of vocabulary indices

        Returns:
            alignments: list of alignments for each sentence pair, i.e. lists of tuples (source_pos, target_pos).
            Alignment positions in sentences start from 1.
        """
        pass


class DiceAligner(BaseAligner):
    def __init__(self, num_source_words: int, num_target_words: int, threshold=0.5):
        self.cooc = np.zeros((num_source_words, num_target_words), dtype=np.uint32)
        self.dice_scores = None
        self.threshold = threshold

    def fit(self, parallel_corpus):
        for sentence in parallel_corpus:
            for source_token in np.unique(sentence.source_tokens):
                for target_token in np.unique(sentence.target_tokens):
                    self.cooc[source_token, target_token] += 1
        self.dice_scores = (2 * self.cooc.astype(np.float32) /
                            (self.cooc.sum(0, keepdims=True) + self.cooc.sum(1, keepdims=True)))

    def align(self, sentences):
        result = []
        for sentence in sentences:
            alignment = []
            for (i, source_token), (j, target_token) in product(
                    enumerate(sentence.source_tokens, 1),
                    enumerate(sentence.target_tokens, 1)):
                if self.dice_scores[source_token, target_token] > self.threshold:
                    alignment.append((i, j))
            result.append(alignment)
        return result


#=========================================================================================================
#================================================================== НАПИСАННАЯ МОДЕЛЬ ВЫРАВНИВАНИЯ =======
#=========================================================================================================


class WordAligner(BaseAligner):
    def __init__(self, num_source_words, num_target_words, num_iters):
        self.num_source_words = num_source_words
        self.num_target_words = num_target_words
        self.translation_probs = np.full((num_source_words, num_target_words), 1 / num_target_words, dtype=np.float32)
        self.num_iters = num_iters

    def _e_step(self, parallel_corpus: List[TokenizedSentencePair]) -> List[np.array]:
        """
        Given a parallel corpus and current model parameters, get a posterior distribution over alignments for each
        sentence pair.

        Args:
            parallel_corpus: list of sentences with translations, given as numpy arrays of vocabulary indices

        Returns:
            posteriors: list of np.arrays with shape (src_len, target_len). posteriors[i][j][k] gives a posterior
            probability of target token k to be aligned to source token j in a sentence i.
        """
        posteriors = []
        for corp in parallel_corpus:
            src = corp.source_tokens
            trg = corp.target_tokens
            tr_probs = self.translation_probs[np.array(src)[:, np.newaxis], trg]
            
            nominator = tr_probs
            denominator = np.sum(tr_probs, axis=0)
            denominator[denominator == 0] = 1
            rates = nominator / denominator
            post = np.fromfunction(lambda j, k: rates[j, k], 
                                   shape=(src.shape[0], trg.shape[0]), dtype=np.int8)
            posteriors += [post]
        return posteriors
            

    def _compute_elbo(self, parallel_corpus: List[TokenizedSentencePair], posteriors: List[np.array]) -> float:
        """
        Compute evidence (incomplete likelihood) lower bound for a model given data and the posterior distribution
        over latent variables.

        Args:
            parallel_corpus: list of sentences with translations, given as numpy arrays of vocabulary indices
            posteriors: posterior alignment probabilities for parallel sentence pairs (see WordAligner._e_step).

        Returns:
            elbo: the value of evidence lower bound
        """
        
        res = float(0)
        for corp, post in zip(parallel_corpus, posteriors):
            src = corp.source_tokens
            trg = corp.target_tokens
            tr_probs = self.translation_probs[src, :][:, trg]
            
            log_g = np.zeros_like(tr_probs)
            log_g[tr_probs > 0] = post[tr_probs > 0] * np.log(tr_probs[tr_probs > 0])
            res += float(np.sum(log_g))
        return res
        

    def _m_step(self, parallel_corpus: List[TokenizedSentencePair], posteriors: List[np.array]):
        """
        Update model parameters from a parallel corpus and posterior alignment distribution. Also, compute and return
        evidence lower bound after updating the parameters for logging purposes.

        Args:
            parallel_corpus: list of sentences with translations, given as numpy arrays of vocabulary indices
            posteriors: posterior alignment probabilities for parallel sentence pairs (see WordAligner._e_step).

        Returns:
            elbo:  the value of evidence lower bound after applying parameter updates
        """
        self.translation_probs[:, :] = 0
        for corp, post in zip(parallel_corpus, posteriors):
            
            src, src_index, src_counts = np.unique(corp.source_tokens, return_counts=True, return_index=True)
            trg, trg_index, trg_counts = np.unique(corp.target_tokens, return_counts=True, return_index=True)
            blank = post[src_index[:, np.newaxis], trg_index]
            blank *= src_counts[:, np.newaxis]
            blank *= trg_counts
            self.translation_probs[src[:, np.newaxis], trg] += blank
        denominator = np.sum(self.translation_probs, axis=1)[:, np.newaxis]
        denominator[denominator == 0] = 1.0
        self.translation_probs /= denominator
        elbo = self._compute_elbo(parallel_corpus, posteriors)
        return elbo
        

    def fit(self, parallel_corpus):
        """
        Same as in the base class, but keep track of ELBO values to make sure that they are non-decreasing.
        Sorry for not sticking to my own interface ;)

        Args:
            parallel_corpus: list of sentences with translations, given as numpy arrays of vocabulary indices

        Returns:
            history: values of ELBO after each EM-step
        """
        history = []
        for i in range(self.num_iters):
            posteriors = self._e_step(parallel_corpus)
            elbo = self._m_step(parallel_corpus, posteriors)
            history.append(elbo)
        return history
    
    def align(self, sentences):
        result = []
        posts = self._e_step(sentences)
        for sentence, post in zip(sentences, posts):
            alignment = []
            post = np.argmax(post, axis=0)
            for i in np.arange(post.shape[0]):
                alignment.append((post[i]+1, i+1))
            result.append(alignment)
        return result

# Вывод теории для написанного ЕМ-алгоритма в классе WordAligner

---

- # E-шаг

Е - шаг заключается в вычислении распределения латентных переменных (в нашем случае - это выравнивания $a_i$) при текущих параметров модели ($\theta_{ij}$). Мы запишем обновление этого распределения через вероятность $p(A|S,T)$, поразумевая, что связь между оригинальными предложениями $S$ и переводом $T$ регулирует матрица вероятностей $\theta$.

Пересчитаем это распределение латентных переменных сначала для одного предложения (считаем, что $n$ - число слов в оригинальном предложении, $m$ - в переводе). По свойству условных вероятностей:

### $$p(A|S,T) = \frac {p(A,T|S)} {p(T|S)}$$

Нужно расписать числитель и знаменатель. Числитель уже расписан в условии задачи:

### $$p(A,T|S)=\prod_{i=1}^m \frac{1}{n}\theta(t_i|s_{a_i})$$

Знаменатель:

#### $ p(T|S) = \sum_{A=(a_1, a_2, ..., a_m)}p(T,A|S) = \sum_{a_1=1}^n \sum_{a_2=1}^n ... \sum_{a_m=1}^n p(T,A|S) = $ { Опять используем равенство в условии через $\theta$} = $\sum_{a_1=1}^n \sum_{a_2=1}^n ... \sum_{a_m=1}^n\prod_{i=1}^m \frac{1}{n}\theta(t_i|s_{a_i}) = \frac{1}{n^m}\sum_{a_1=1}^n \sum_{a_2=1}^n ... \sum_{a_m=1}^n \prod_{i=1}^m \theta(t_i|s_{a_i}) = $ {По свойствам дистрибутивности выносим $\theta(t_i, s_j)$ за знак суммы всех произведений перестановок оставшихся элементов матрицы $\theta({t_i, s_j}$, и так для каждого элемента, после этого объединим суммы по свойству вероятностей} = $\frac{1}{n^m}\prod_{i=1}^m\sum_{j=1}^n\theta(t_i, s_j)$, так как $\sum_{j=1}^n\theta(t_i, s_j)=1$.

Внесём их в одну искомую дробь:

### $$p(A|S,T) = \frac {p(A,T|S)} {p(T|S)} = \frac {\frac{1}{n^m}\prod_{i=1}^m \theta(t_i|s_{a_i})} {\frac{1}{n^m}\prod_{i=1}^m\sum_{j=1}^n \theta(t_i|s_{a_j})} = \prod_{i=1}^m \frac {\theta(t_i|s_{a_i})} {\sum_{j=1}^n \theta(t_i|s_{a_j})}$$

Совершим переход от одного предложения к нескольким. По сути указанный пересчёт нужно произвести для каждого предложения корпуса. В наших обозначениях $R$ - число предложений корпуса, $n_r$, $m_r$ - число слов в оригинальном предложении и переводе, $r=1,2,...,R$. Тогда:

### $$p(A_r| S_r, T_r) = \prod_{i=1}^{m_r}\frac {\theta(t_i|s_{a_i})} {\sum_{j=1}^{n_r} \theta(t_i|s_{a_j})}$$

В этом обновлении распределения латентных переменных и заключён Е-шаг алгоритма. В дальнейшем для отдельного предложения обозначим за $g_{ij}$ - полученную апостериорную вероятность для пары $(t_i, s_j)$, то есть:

### $$g_{ij} = \frac {\theta(t_i|s_{a_i})} {\sum_{j=1}^{n} \theta(t_i|s_{a_j})}$$

---

---

---

- # M-шаг


M - шаг: максимизация правдоподобия по параметру $\theta$:

### $$Maximize\ \ \ \int p(A) log \frac{p(A, S, T| \theta)}{p(A)}dA\ \ \ by\ \ \theta$$

Или, что эквивалентно написанному выше:

### $$Maximize\ \ \  \mathbb{E}_{a_i \sim p(A)}\prod_i log(p(a_i, t_i, s_i| \theta)) \ \ \ by \ \ \theta$$

Распишем через параметр вероятностей $\theta$ (воспользуемя ещё раз свойтвами условной вероятности и представлением через $g_{ij}$, а так же раскроем логарифм произведения в сумму логарифмов):

### $$L = \sum_{i=1}^m\sum_{j=1}^ng_{ij} log \theta(t_i, s_j) $$

Это выражение - ELBO, оно же и расписанное в данной задаче максимизируемое математическое ожидание (для одного предложения! Для всех предложений при максимизации нужно проссумировать эти функции по корпусу, добавится ещё одна сумма по корпусам, и индексы изменятся для каждого предложения отдельно). Учтём, что макимизация является условной: все столбцы $\theta$ должны "суммироваться" в единицу, так как по условию задачи каждое слово перевода соответствует некоторому слову исходного языка. Значит, событие "$t_i$ является переводом хотя бы одного слова ихсодного языка" - достоверно, то есть $\sum_{i=1}^m\theta(t_i, s_j) = 1,\ \ \ j = 1, 2, ..., n$ ($n$ условий). 

Для выполнения M-шага (максимизации) рапишем лагранжиан данной функции:

### $$Lagr = \sum_{i=1}^m\sum_{j=1}^ng_{ij} log \theta(t_i, s_j) + \sum_{j=1}^n \lambda_{j}(1 - \sum_{i=1}^m\theta(t_i, s_j))$$

Продифференцируем по двойственным переменным $\lambda_j$:

### $$\frac{\partial Lagr}{\partial \theta(t_i, s_j)} = \frac{g_{ij}}{\theta(t_i, s_j)} - \lambda_j = 0$$

Тогда обновлённые значения параметров матрицы $\theta$ выражаются через двойственные переменные следующим образом:

### $$\theta(t_i, s_j) = \frac{g_{ij}}{\lambda_j}$$

Подставим это в лагранжиан:

### $$L = \sum_{i=1}^m\sum_{j=1}^n g_{ij} log \frac{g_{ij}}{\lambda_j} + \sum_{j=1}^n \lambda_j(1 - \sum_{i=1}^m \frac{g_{ij}}{\lambda_j}) = \sum_{i=1}^m\sum_{j=1}^n g_{ij} (log \frac{g_{ij}}{\lambda_j} - 1) + \sum_{j=1}^n \lambda_j$$

Отдельно отметим, что полная максимизируемая функция является суммой данных L для каждого корпуса. Но для того, чтобы произвести М-шаг, можно разделить их на разные получаемые параметры для корпусов.
Эта задача - так же задача оптимизации, но уже безусловная, на переменные $\lambda_j$. Найдём их, продифференцировав полученный лагранжиан по ним:

### $$\frac{\partial Lagr}{\partial \lambda_j} = -\frac{1}{\lambda_j}\sum_{i=1}^n g_{ij} + 1 = 0$$

Выразим $\lambda_j$:

### $$\lambda_j = \sum_{i=1}^m g_{ij}$$

И, наконец, обновлённые значения для $\theta$ следующие:

### $$\theta(t_i, s_j) = \frac{g_{ij}}{\sum_{i=1}^m g_{ij}}$$

В этом обновлении заключается М-шаг ($g_{ij}$ расписано в конце E-шага). Отметим, что элементы матрицы $\theta$ обновляются для каждого предложения по-отдельности.

---

- # ELBO

Теперь выведем формулу для вычисления самой ELBO. Для этого достаточно взять первую часть расписанного лагранжиана - максимизируемого на М-шаге математического ожидания и подставить в неё найденные обновлённые параметры $\theta$. Дополнительно к этому для полного подсчёта ELBO нужно аккумулировать сумму по всем корпусам.

### $$ELBO = \sum_{r=1}^R \sum_{i=1}^{m_r} \sum_{j=1}^{n_r} g_{ij}^r ( log(\theta(t_{r(i)}, s_{r(j)}) - log(\sum_{i=1}^m g_{ij}^r))$$

Здесь для того, чтобы "обработать" один корпус (одно внешнее слагаемое ELBO) из матрицы $\theta$ берутся элементы, соответствующие парам слов в соответствующем предложении. $i$, $j$ - порядковые номера этих слов в предложении, а за $r(i)$ и $r(j)$ обозначены отображения этих слов в порядковые номера во всей матрице $\theta$. Для апостериорных вероятностей $g_{ij}$ добавляется индекс, так как они считаютя на E-шаге для каждого предлложения отдельно. Здесь уже не проводятся выкладки, так как все они были получены ранее, здесь же достаточно расписать способ извлечения элементов матрицы $\theta$ для алгоритма, что и было сделано.